+[xlsxwriter](https://xlsxwriter.readthedocs.io/index.html)

In [23]:
import pandas as pd
import numpy as np
import ConnectDB as cn
from datetime import datetime
import xlsxwriter 
from xlsxwriter.utility import xl_rowcol_to_cell,xl_cell_to_rowcol

In [24]:
#Time Index
Fcst_Day=pd.date_range('2019/03/01','2019/03/10',freq='D')
BLSP_Day=pd.date_range('2019/03/01','2019/03/10',freq='2D')

In [25]:
Fcst_Day

DatetimeIndex(['2019-03-01', '2019-03-02', '2019-03-03', '2019-03-04',
               '2019-03-05', '2019-03-06', '2019-03-07', '2019-03-08',
               '2019-03-09', '2019-03-10'],
              dtype='datetime64[ns]', freq='D')

In [26]:
BLSP_Day

DatetimeIndex(['2019-03-01', '2019-03-03', '2019-03-05', '2019-03-07',
               '2019-03-09'],
              dtype='datetime64[ns]', freq='2D')

In [27]:
FcQty=pd.DataFrame(np.random.rand(10),index=Fcst_Day,columns=['FcQty']).reset_index()
BLQty=pd.DataFrame(np.random.rand(5),index=BLSP_Day,columns=['BLQty']).reset_index()

In [28]:
FcQty

,index,FcQty
0,2019-03-01,0.449466
1,2019-03-02,0.097231
2,2019-03-03,0.116070
3,2019-03-04,0.897400
4,2019-03-05,0.987979
5,2019-03-06,0.851414
6,2019-03-07,0.229453
7,2019-03-08,0.708699
8,2019-03-09,0.515176
9,2019-03-10,0.014910


In [29]:
FcQty=FcQty.rename(columns={'index':'Day'})
BLQty=BLQty.rename(columns={'index':'Day'})

In [30]:
dfMain=pd.merge(FcQty,BLQty,how='outer',left_on='Day', right_on='Day')

In [31]:
dfMain

,Day,FcQty,BLQty
0,2019-03-01,0.449466,0.880221
1,2019-03-02,0.097231,NaN
2,2019-03-03,0.116070,0.238112
3,2019-03-04,0.897400,NaN
4,2019-03-05,0.987979,0.153174
5,2019-03-06,0.851414,NaN
6,2019-03-07,0.229453,0.413905
7,2019-03-08,0.708699,NaN
8,2019-03-09,0.515176,0.911314
9,2019-03-10,0.014910,NaN


In [32]:
def dataInterval(data1):
    #d1 =datetime.strptime(data1, '%Y-%m-%d')
    d1=data1.strftime('%Y/%m/%d')
    return d1    

In [33]:
#dfMain.loc[:,'dfMain']
dfMain['Day'] = dfMain['Day'].apply(dataInterval)  

In [34]:
dfMain=dfMain.set_index('Day')
dfMain.head(3)

,FcQty,BLQty
Day,,
2019/03/01,0.449466,0.880221
2019/03/02,0.097231,NaN
2019/03/03,0.116070,0.238112


In [35]:
dfMain=dfMain.fillna(0)
dfMain

,FcQty,BLQty
Day,,
2019/03/01,0.449466,0.880221
2019/03/02,0.097231,0.000000
2019/03/03,0.116070,0.238112
2019/03/04,0.897400,0.000000
2019/03/05,0.987979,0.153174
2019/03/06,0.851414,0.000000
2019/03/07,0.229453,0.413905
2019/03/08,0.708699,0.000000
2019/03/09,0.515176,0.911314


In [36]:
dfMain['Gap']=dfMain['FcQty']-dfMain['BLQty']

In [37]:
#轉置
dfMain=dfMain.T

In [38]:
dfMain=dfMain.reset_index()

In [39]:
dfMain

Day,index,2019/03/01,2019/03/02,2019/03/03,2019/03/04,2019/03/05,2019/03/06,2019/03/07,2019/03/08,2019/03/09,2019/03/10
0,FcQty,0.449466,0.097231,0.116070,0.8974,0.987979,0.851414,0.229453,0.708699,0.515176,0.01491
1,BLQty,0.880221,0.000000,0.238112,0.0000,0.153174,0.000000,0.413905,0.000000,0.911314,0.00000
2,Gap,-0.430755,0.097231,-0.122042,0.8974,0.834805,0.851414,-0.184452,0.708699,-0.396139,0.01491


In [40]:
#Excel 存檔
EndDate=datetime.now()
EndDate=EndDate.strftime('%Y/%m/%d')
FileName='FCST_Gap'+EndDate.replace('/','')+'.xlsx'
workbook = xlsxwriter.Workbook(FileName)

In [41]:
# 儲存格格式
cell_Time=workbook.add_format({'bg_color':'#C61031','font_name':'Calibri','bold': True,
                               'font_color':'#FFFFFF','num_format': 'yy/mm/dd'})
cell_format_Number = workbook.add_format({'num_format':'(#,##0.00);[RED](#,##0.00)'})

In [42]:
# 新增 Sheet
Sheet1 = workbook.add_worksheet('ModelGroupSummary')
#新增表頭
for col_num, value in enumerate(dfMain.columns.values):
    Sheet1.write(0, col_num + 1, value, cell_Time)
    #print(0, col_num + 1,value,)

+ dfMain.shape[0] 幾筆資料
+ dfMain.iloc[row,:] 一筆資料所有欄位

In [44]:
#Table 資料
for row in range(0,dfMain.shape[0]):
    for col, value in enumerate(dfMain.iloc[row,:]):
        print(col+1,row+1,cName,value)
        #取得儲存格名稱
        cName=xl_rowcol_to_cell(row+1,col+1) 
        #取得儲存格座標
        r1,c1=xl_cell_to_rowcol(cName)   
        cRow_1=xl_rowcol_to_cell(r1-1,c1)  
        cRow_2=xl_rowcol_to_cell(r1-2,c1)       
        # 第 3 列 Gap 寫入公式
        if (row+1)==3:
            if (cName)=='B4':               
                Sheet1.write(cName,value)
            else:
                cal='='+cRow_1+'-'+cRow_2
                Sheet1.write(cName,cal,cell_format_Number)
                print(cName,cal)       
        else:
            Sheet1.write(cName,value)
#公式函數 
Sheet1.write('A2','=SUM(C2:L2)')          
workbook.close()        

1 1 L4 FcQty
2 1 B2 0.4494664941953789
3 1 C2 0.09723110407135316
4 1 D2 0.11606958444673654
5 1 E2 0.8973999377314037
6 1 F2 0.9879785189708049
7 1 G2 0.8514139984305261
8 1 H2 0.2294532619851355
9 1 I2 0.7086994857478841
10 1 J2 0.5151757515002819
11 1 K2 0.014910024826723767
1 2 L2 BLQty
2 2 B3 0.8802211616516774
3 2 C3 0.0
4 2 D3 0.23811198636590902
5 2 E3 0.0
6 2 F3 0.15317401495974325
7 2 G3 0.0
8 2 H3 0.41390485050548254
9 2 I3 0.0
10 2 J3 0.9113143770564557
11 2 K3 0.0
1 3 L3 Gap
2 3 B4 -0.43075466745629853
C4 =C3-C2
3 3 C4 0.09723110407135316
D4 =D3-D2
4 3 D4 -0.12204240191917248
E4 =E3-E2
5 3 E4 0.8973999377314037
F4 =F3-F2
6 3 F4 0.8348045040110617
G4 =G3-G2
7 3 G4 0.8514139984305261
H4 =H3-H2
8 3 H4 -0.18445158852034704
I4 =I3-I2
9 3 I4 0.7086994857478841
J4 =J3-J2
10 3 J4 -0.39613862555617374
K4 =K3-K2
11 3 K4 0.014910024826723767
L4 =L3-L2
